<a href="https://colab.research.google.com/github/pstansfeld/MemProtMD/blob/main/PDB_fix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/pstansfeld/MemProtMD/raw/main/mr-membrane-protein.png" height="200" align="right" style="height:240px">

##Clean a protein structure for use with MemProtMD

This Notebook uses [PDB2PQR](https://onlinelibrary.wiley.com/doi/10.1002/pro.3280), [available here](https://github.com/Electrostatics/pdb2pqr), followed by energy minimisation using [GROMACS](https://doi.org/10.1016/j.softx.2015.06.001).



In [7]:
#@title Upload a PDB coordinate file
import os
import sys
from google.colab import files

sys.path.append("/usr/local/lib/python3.7/site-packages/")
os.chdir("/content/")

uploaded_files = files.upload()
filename = list(uploaded_files.keys())[0]
name, ext = os.path.splitext(filename)
working_dir = f"/content/{name}/"
os.makedirs(working_dir, exist_ok=True)
os.rename(filename, working_dir + filename)
os.chdir(working_dir)

Saving ecMraY.pdb to ecMraY (2).pdb


In [8]:
#@title Install dependencies
%%capture

if not os.path.isdir("/content/content/gromacs/"):
    !apt-get update -y
    !pip install pdb2pqr GromacsWrapper MDAnalysis py3dmol
    !wget https://raw.githubusercontent.com/pstansfeld/MemProtMD/master/gromacs.zip
    !unzip -o gromacs.zip
    !ln -s /content/content/gromacs/bin/gmx /usr/bin/gmx
    !ln -s /content/content/gromacs/bin/gmx_d /usr/bin/gmx_d

import gromacs
import MDAnalysis
import glob
import shutil
import py3Dmol
from gromacs import pdb2gmx, editconf, grompp, mdrun, trjconv

def py3dmol_view(pdbfilename, working_dir):
    mview = py3Dmol.view(width=800, height=400)
    with open(working_dir + pdbfilename, "r") as f:
        mol1 = f.read()
    mview.addModel(mol1, "pdb")
    mview.setStyle({"cartoon": {"color": "spectrum"}})
    mview.setBackgroundColor("0xffffff")
    mview.zoomTo()
    mview.show()

def energy_minimize(filename, working_dir):
    if not os.path.exists("temp"):
        os.makedirs("temp")
    os.chdir("temp")
    os.system(f"pdb2pqr30 --ff CHARMM --keep-chain {working_dir + filename} pqr.pdb")

    pdb2gmx(f="pqr.pdb", ignh=True, ff="charmm27", water="tip3p", o="conf.pdb")
    editconf(f="conf.pdb", d=8, c=True, o="conf.pdb")

    with open("em.mdp", "w") as em:
        em.write("integrator = steep\nnsteps = 5000\nemtol = 100\nemstep = 0.001")

    grompp(f="em.mdp", maxwarn=5, o="em", c="conf.pdb")
    mdrun(deffnm="em", c="clean.pdb")

    trjconv(f="clean.pdb", o=working_dir + f"fixed-{filename}", s="em.tpr", input=("system"))

    os.chdir("..")
    shutil.rmtree("temp", ignore_errors=True)

In [9]:
#@title View Input
py3dmol_view(filename, working_dir)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [10]:
#@title Energy Minimise
%%capture
energy_minimize(filename, working_dir)

In [12]:
#@title View & Download
py3dmol_view(f'fixed-{filename}', working_dir)
files.download(working_dir + 'fixed-'+filename)

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>